In [4]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Evaluation.K_FoldEvaluator import K_FoldEvaluator
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [5]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()

In [6]:
URM_stacked = combine_matrices(ICM=ICM,URM=URM)

In [7]:
rp3_recommender=RP3betaRecommender(URM_train=URM_stacked)
rp3_recommender.fit(topK=181, 
                    alpha=0.5709402717259106, 
                    beta=0.3566066452521787, 
                    implicit=True, 
                    normalize_similarity=True)

In [8]:
slime_recommender=MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
slime_recommender.fit(topK=420, 
                      l1_ratio=0.06061216785568925, 
                      alpha=0.00554982938879437, 
                      workers = 20)

100%|████████████████████████████████████▉| 18000/18059 [12:27<00:01, 45.70it/s]

In [9]:
sim_slime = slime_recommender.W_sparse
sim_rp3 = rp3_recommender.W_sparse
    
similarity_recommender=ItemKNNSimilarityHybridRecommender(URM_train=URM_stacked,
                                                                     Similarity_1=sim_rp3,
                                                                     Similarity_2=sim_slime)
similarity_recommender.fit(topK=int(524.9285577442362), 
                           alpha=0.16115285930157153)

In [10]:
IALS_recommender=IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=10, 
                     num_factors=42, 
                     alpha=0.7617528864750021, 
                     reg=8.926401306541349)

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 7.66 sec


100%|████████████████████████████████████▉| 18056/18059 [12:40<00:00, 45.70it/s]

IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 15.35 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 23.38 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 31.06 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 39.80 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 48.19 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 59.05 sec
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.12 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.25 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.42 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.42 min


In [ ]:
recommender = MergeTwoModelsByScores(URM_train=URM_stacked,recommenders=[similarity_recommender,IALS_recommender])
recommender.fit(alpha=0.8481117477716246,beta=0.34145203215269804)

In [ ]:
submission = create_submission(recommender)
write_submission(submission,"hybrid_KFOLD_BEST25")